In [1]:
import pandas as pd
import numpy as np
#!pip install tqdm
import tqdm 
#from tqdm.auto
#import trange
import nltk
import re
data = pd.read_csv('E:\project university\Выпускная работа\Сlean.csv')

In [5]:
!pip install tqdm


In [2]:
texts=data['description']

In [3]:
type(texts)

pandas.core.series.Series

In [4]:
#Предобработка текста
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")
def token_and_stem(text):
    tokens=[word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens=[]
    for token in tokens:
        if re.search('[а-яА-Я]', token):
            filtered_tokens.append(token)
    stems=[stemmer.stem(t) for t in filtered_tokens]
    return stems


In [17]:
stopwords = nltk.corpus.stopwords.words('russian')
#расширяю список стоп-слов
stopwords.extend(['что','это','так', 'вот', 'быть','как','в', 'к','на', 'спешите', 'внимание', 'лучшее', 'самое','Успейте','выгодно', 'Прекрасная'])

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

n_featur=200000
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000,
                                 min_df=0.01, stop_words=stopwords,
                                 use_idf=True, tokenizer=token_and_stem, ngram_range=(1,3))

In [18]:
%%time
tfidf_matrix=tfidf_vectorizer.fit_transform(texts)

C:\Users\Daniel\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['бол', 'больш', 'будт', 'быт', 'вед', 'вниман', 'впроч', 'всег', 'всегд', 'выгодн', 'даж', 'друг', 'е', 'ег', 'ем', 'есл', 'ест', 'ещ', 'зач', 'зде', 'ил', 'иногд', 'когд', 'конечн', 'куд', 'лучш', 'межд', 'мен', 'мног', 'мо', 'можн', 'нег', 'нельз', 'нибуд', 'никогд', 'нич', 'опя', 'посл', 'пот', 'почт', 'прекрасн', 'разв', 'сво', 'себ', 'совс', 'спеш', 'теб', 'тепер', 'тог', 'тогд', 'тож', 'тольк', 'усп', 'хорош', 'хот', 'чег', 'чут', 'эт'] not in stop_words.
  warnings.warn(


Wall time: 16min 18s


In [19]:
type(tfidf_matrix)
tfidf_matrix

<100723x2743 sparse matrix of type '<class 'numpy.float64'>'
	with 10074824 stored elements in Compressed Sparse Row format>

In [20]:
print(tfidf_matrix.shape)

(100723, 2743)


In [21]:
tfidf_matrix[:10]

<10x2743 sparse matrix of type '<class 'numpy.float64'>'
	with 372 stored elements in Compressed Sparse Row format>

In [22]:
num_clusters = 5

# Метод к-средних - KMeans
from sklearn.cluster import KMeans
km = KMeans(n_clusters=num_clusters)

In [23]:
%%time
km.fit(tfidf_matrix)

Wall time: 5min 37s


KMeans(n_clusters=5)

In [24]:
%%time
idx = km.fit(tfidf_matrix)
clusters = km.labels_.tolist()


Wall time: 5min 19s


In [25]:
len(km.labels_)
clusters[:10]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [26]:
clusterkm = km.labels_.tolist()
frame = pd.DataFrame(texts)

#k-means
out = { 'text': texts, 'cluster': clusterkm, 'description': data['description'] }
frame1 = pd.DataFrame(out, columns = ['text', 'cluster', 'description'])

In [28]:
frame1.head(10)

,text,cluster,description
0,Прекрасная однокомнатная квартира в доме с лиф...,3,Прекрасная однокомнатная квартира в доме с лиф...
1,"Светлая, просторная комната 19 кв. метров. Отд...",3,"Светлая, просторная комната 19 кв. метров. Отд..."
2,"Первомайская, 15 мин. пешкомТёплая уютная квар...",3,"Первомайская, 15 мин. пешкомТёплая уютная квар..."
3,"в шаговой доступности парк Сокольники. , Разви...",3,"в шаговой доступности парк Сокольники. , Разви..."
4,Ильинское с.Все коммуникации центральные.Кварт...,3,Ильинское с.Все коммуникации центральные.Кварт...
5,Продается срочно однокомнатная квартира под ре...,3,Продается срочно однокомнатная квартира под ре...
6,"Знамя Октября пос.Продается светлая, уютная, п...",3,"Знамя Октября пос.Продается светлая, уютная, п..."
7,"Маяковская, 10 мин. пешкомБаррикадная, 10 мин....",3,"Маяковская, 10 мин. пешкомБаррикадная, 10 мин...."
8,Продается двухкомнатная квартира общей площадь...,3,Продается двухкомнатная квартира общей площадь...
9,Домодедово г.Успейте купить выгодно!!! ...,3,Домодедово г.Успейте купить выгодно!!! ...
